In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



# Import des modules contenant les fonctions utilitaires
import src.data_helpers as dth

In [ ]:
# Réglage des graphiques
plt.style.use('seaborn-whitegrid')

plt.rc('font', size=12)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=12)

dims_fig = (25,20)

In [ ]:
customers = pd.read_csv('data/olist_customers_dataset.csv', sep=',')
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv', sep=',')
order_items = pd.read_csv('data/olist_order_items_dataset.csv', sep=',')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv', sep=',')
order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv', sep=',')
orders = pd.read_csv('data/olist_orders_dataset.csv', sep=',')
products = pd.read_csv('data/olist_products_dataset.csv', sep=',')
sellers = pd.read_csv('data/olist_sellers_dataset.csv', sep=',')

In [ ]:
order_payments_agg = order_payments.groupby('order_id').agg({'payment_type': 'first', 'payment_installments': 'first', 'payment_value': 'sum'})
order_reviews_agg = order_reviews.groupby('order_id').agg('mean')
geolocation_agg = geolocation.groupby('geolocation_zip_code_prefix').agg('mean')
order_items_agg = order_items.groupby('order_id').agg({'order_id': 'first', 'product_id': 'first', 'seller_id': 'first', 'shipping_limit_date': 'first', 'price': 'sum', 'freight_value': 'sum'})

In [ ]:
orders_merge = orders.merge(order_payments_agg, how="left", on="order_id", validate="1:m").merge(order_reviews_agg, how="left", on="order_id", validate="m:1")

In [ ]:
sellers_merge = sellers.merge(geolocation_agg, how="left", left_on= ['seller_zip_code_prefix'], right_on=['geolocation_zip_code_prefix'])
sellers_merge.rename(columns={"geolocation_lat": "seller_geolocation_lat", "geolocation_lng": "seller_geolocation_lng"}, inplace=True)

In [ ]:
order_items_merge = order_items_agg.merge(products, how="left", on="product_id", validate="m:1").merge(sellers_merge, how="left", on="seller_id", validate="m:1")

In [ ]:
customers_merge = customers.merge(geolocation_agg, how="left", left_on= ['customer_zip_code_prefix'], right_on=['geolocation_zip_code_prefix'])
customers_merge.rename(columns={"geolocation_lat": "customer_geolocation_lat", "geolocation_lng": "customer_geolocation_lng"}, inplace=True)

In [ ]:
orders_merge = orders_merge.merge(customers_merge, how="left", on="customer_id", validate="m:1").merge(order_items_merge, how="left", on="order_id", validate="1:1")

In [ ]:
orders_merge.to_csv('merge.csv', index=False)